<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801/dataset-filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install git-lfs
from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


## Loading Dataset

In [4]:
!pip install datasets

In [5]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted")

## Cosine Similarity / Euclidean Distance

In [53]:
!pip install scipy

In [54]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from scipy.spatial import distance
from datasets import load_dataset
from tqdm import tqdm

# Load tokenizer and model (e.g., multilingual BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed_sentence(sentence, lang):
    tokens = tokenizer(sentence, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        embeddings = model(**tokens)
    return embeddings.last_hidden_state.mean(dim=1)

In [ ]:
# Initialize variables to store the maximum lengths
max_length_arabic = 0
max_length_english = 0

# Iterate over the dataset to find the maximum lengths
for row in dataset['train']:
    length_arabic = len(row['arabic'].split())  # Number of words in the Arabic sentence
    length_english = len(row['english'].split())  # Number of words in the English sentence

    # Update the maximum lengths if current lengths are greater
    if length_arabic > max_length_arabic:
        max_length_arabic = length_arabic
    if length_english > max_length_english:
        max_length_english = length_english

if max_length_arabic > max_length_english:
  max_length = max_length_arabic
else:
  max_length = max_length_english

print(max_length)

449


In [55]:
import numpy as np

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

def calculate_euclidean_distance(embedding1, embedding2):
    # Flatten embeddings to ensure they are 1-D
    embedding1 = np.array(embedding1).flatten()
    embedding2 = np.array(embedding2).flatten()
    return distance.euclidean(embedding1, embedding2)

In [ ]:
# Initialize an empty list to store filtered data
filtered_data_cosine = []

# Define a similarity threshold
similarity_threshold = 0.6  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    similarity = calculate_cosine_similarity(arabic_embedding, english_embedding)
    if similarity > similarity_threshold:
        filtered_data_cosine.append(row)

In [ ]:
from datasets import Dataset

# Assuming filtered_data is a list of dictionaries
# Convert the filtered data to a dictionary format
data_dict = {"arabic": [row['arabic'] for row in filtered_data_cosine],
             "english": [row['english'] for row in filtered_data_cosine]}

# Create a new dataset from the dictionary
filtered_dataset_6 = Dataset.from_dict(data_dict)

# Now filtered_dataset is a Hugging Face Dataset containing your filtered data

In [8]:
filtered_dataset

Dataset({
    features: ['arabic', 'english'],
    num_rows: 7950
})

In [11]:
filtered_dataset_5

Dataset({
    features: ['arabic', 'english'],
    num_rows: 6489
})

In [14]:
filtered_dataset_6

Dataset({
    features: ['arabic', 'english'],
    num_rows: 2771
})

In [20]:
dataset['train'] = filtered_dataset_5

In [21]:
dataset.push_to_hub("arb-eng-parallel-10k-splitted-cosine-5")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [74]:
# Initialize an empty list to store filtered data
filtered_data_euclidean = []

# Define a similarity threshold
distance_threshold = 9.5  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    euclidean_dist = calculate_euclidean_distance(arabic_embedding, english_embedding)
    if euclidean_dist < distance_threshold:
        filtered_data_euclidean.append(row)

Processing: 100%|██████████| 8000/8000 [14:11<00:00,  9.40row/s]


In [75]:
# Assuming filtered_data is a list of dictionaries
# Convert the filtered data to a dictionary format
data_dict = {"arabic": [row['arabic'] for row in filtered_data_euclidean],
             "english": [row['english'] for row in filtered_data_euclidean]}

# Create a new dataset from the dictionary
filtered__euclidean_dataset_95 = Dataset.from_dict(data_dict)

In [38]:
filtered__euclidean_dataset_5

Dataset({
    features: ['arabic', 'english'],
    num_rows: 77
})

In [41]:
filtered__euclidean_dataset_9

Dataset({
    features: ['arabic', 'english'],
    num_rows: 78
})

In [44]:
filtered__euclidean_dataset_30

Dataset({
    features: ['arabic', 'english'],
    num_rows: 106
})

In [47]:
filtered__euclidean_dataset_100

Dataset({
    features: ['arabic', 'english'],
    num_rows: 6636
})

In [58]:
filtered__euclidean_dataset_80

Dataset({
    features: ['arabic', 'english'],
    num_rows: 666
})

In [70]:
filtered__euclidean_dataset_90

Dataset({
    features: ['arabic', 'english'],
    num_rows: 3141
})

In [76]:
filtered__euclidean_dataset_95

Dataset({
    features: ['arabic', 'english'],
    num_rows: 5227
})

In [77]:
dataset['train'] = filtered__euclidean_dataset_95

In [78]:
dataset.push_to_hub("arb-eng-parallel-10k-splitted-euclidean-95")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

## Machine Translation and Monolingual Similarity

In [ ]:
!pip install transformers torch sentencepiece

In [7]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-ar-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [21]:
def truncate_tokens(tokenizer, text, max_length):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
    return tokenizer.decode(tokens)

# Update the translate function
def translate(text):
    truncated_text = truncate_tokens(tokenizer, text, 512)
    inputs = tokenizer(truncated_text, return_tensors="pt", padding=True)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [ ]:
from datasets import Dataset

# Assuming you are working with the 'train' split
train_dataset = dataset['train']

# Convert to a list of dictionaries
dataset_list = train_dataset.to_dict()

# Remove the problematic row
index_to_remove = 7413
for key in dataset_list:
    dataset_list[key].pop(index_to_remove)

# Create a new dataset from the modified list
filtered_train_dataset = Dataset.from_dict(dataset_list)

# Update the DatasetDict with the new dataset
dataset['train'] = filtered_train_dataset

In [22]:
import torch

# Add the translated text as a new column
translated_dataset = dataset.map(lambda example: {'translated': translate(example['arabic'])})

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [24]:
translated_dataset.push_to_hub("arb-eng-parallel-10k-splitted-translated-arabic")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
translated_dataset

DatasetDict({
    train: Dataset({
        features: ['arabic', 'english', 'translated'],
        num_rows: 7999
    })
    validation: Dataset({
        features: ['arabic', 'english', 'translated'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['arabic', 'english', 'translated'],
        num_rows: 1000
    })
})

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from scipy.spatial import distance
from datasets import load_dataset
from tqdm import tqdm

# Load tokenizer and model (e.g., multilingual BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed_sentence(sentence, lang):
    tokens = tokenizer(sentence, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        embeddings = model(**tokens)
    return embeddings.last_hidden_state.mean(dim=1)

import numpy as np

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [49]:
# Initialize an empty list to store filtered data
filtered_data_cosine = []

# Define a similarity threshold
similarity_threshold = 0.95  # Adjust this based on your needs

for row in tqdm(translated_dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')
    translated_embedding = embed_sentence(row['translated'], 'tr')

    similarity = calculate_cosine_similarity(translated_embedding, english_embedding)
    if similarity > similarity_threshold:
        filtered_data_cosine.append(row)

Processing: 100%|██████████| 7999/7999 [21:28<00:00,  6.21row/s]


In [50]:
from datasets import Dataset

data_dict = {"arabic": [row['arabic'] for row in filtered_data_cosine],
             "english": [row['english'] for row in filtered_data_cosine]}

filtered_dataset_95 = Dataset.from_dict(data_dict)

In [32]:
filtered_dataset_4

Dataset({
    features: ['arabic', 'english'],
    num_rows: 7931
})

In [36]:
filtered_dataset_2

Dataset({
    features: ['arabic', 'english'],
    num_rows: 7998
})

In [39]:
filtered_dataset_10

Dataset({
    features: ['arabic', 'english'],
    num_rows: 290
})

In [42]:
filtered_dataset_075

Dataset({
    features: ['arabic', 'english'],
    num_rows: 7389
})

In [45]:
filtered_dataset_085

Dataset({
    features: ['arabic', 'english'],
    num_rows: 6681
})

In [48]:
filtered_dataset_9

Dataset({
    features: ['arabic', 'english'],
    num_rows: 5736
})

In [51]:
filtered_dataset_95

Dataset({
    features: ['arabic', 'english'],
    num_rows: 3666
})

In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['arabic', 'english'],
        num_rows: 3666
    })
    validation: Dataset({
        features: ['arabic', 'english'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['arabic', 'english'],
        num_rows: 1000
    })
})

In [53]:
dataset['train'] = filtered_dataset_95

In [55]:
# dataset.push_to_hub("arb-eng-parallel-10k-splitted-machine-translation-cosine-5")

dataset.push_to_hub("arb-eng-parallel-10k-splitted-translated-arabic-cosine-95")
# machine_translation

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]